# 1) Background

### Outline of Algorithm: 

The paper I have chosen describes a type of finite mixture algorithm, one for modeling patterns of an outcome of interest as clusters across time. The method is termed by the authors as Group-based Latent Trajectory Modeling, and was developed by Daniel Nagin in 1999.

The general specification for the group based latent trajectory models is provided in equations (1) and (2). Equation (1) describes the basic form of the finite mixture model--i.e., summing a finite number of latent groupings believed to compose the underlying population. Since group membership is not observed, the proportion of the $j$ underlying population belonging to each of the latent trajectory groups $\pi_j$, must be estimated. This requires the aggregation of the $J$ conditional likelihood functions, forming the unconditional probability of the data, $Y_i$ (Nagin 2005).


$$P(Y_i) = \sum_{j}^J \pi_j P^j (Y_i)\tag{1}$$ 

Here, $P(Y_i)$ estimates the unconditional probability of seeing the trajectory of the outcome measure for individual $i$. 

The key to the "group-based" approach is an underlying idea called "conditional independence." The likelihood function this produces is denoted in equation (2). For a give group $j$, conditional independence makes the assumption that the distribution of $y_{it}$ is independent of the observed value of the outcome in prior periods, $y_{it-1}$,  $y_{it-2}$,... This assumption helps to reduce the complexity of the model, and when combined with the EM algorithm, allows for identification of latent trajectory groupingsdespite the complex nature of the model and data.

$$L = \prod^N P(Y_i)\tag{2}$$



## Readme for Python Module: 

### Group-Based Finite Mixture Models: A Latent Trajectory Approach 

**Code for course project use only at this time.**

Final project for STA 663 
* Takes time variable and continuous outcome 
* Generates latent trajectories (time-dependent clusters) of outcome behavior over time 

**Dependencies**

Required

* Python (created using 2.7.x)
* Numpy
* Scipy 
* Pymix (https://github.com/klahnakoski/pymix.git)

**Recommended**

* ggplot (for trajectory plotting function included in the repository)
* pandas (for trajectory plotting function included in the repository)
* STAN (for Bayesian alternative specification)
* pystan (for Bayesian alternative specification)

## 2) Implementation

In [5]:
from __future__ import division
import os
import sys
import glob
from mixture import *
import mixture
import numpy as np
import pandas as pd
from ggplot import *
import pandas as pd

# ------- #
# Normal Mixture #
# ------- #
# Normal Distribution Functions 

# Generate Means of Normal Mixture
def norm_means(data): 
    cols = data.shape[1]
    return np.repeat(0.0,cols) 

# Generate Covariance Structure of Normal Mixture
def norm_cov(data):
    m = data.shape[1]
    covs = np.repeat(1.0,m) 
    return np.diag(covs)

# Single Multivariate Normal distribution function
def norm_singdist(ng, data):
    m = data.shape[1]
    dist = mixture.MultiNormalDistribution(m, 
        norm_means(data),norm_cov(data))
    return dist

# Generalize to multiple clusters of multivariate distribution function
def norm_multdist(ng, data):
    temp = list()
    for x in range(ng):
        temp.append(norm_singdist(2, TO1adj))
    return temp

# Data functions
def build_mix_dat(data):
    mixdat = mixture.DataSet()
    mixdat.fromArray(data)
    return mixdat

# Normal Mixture Model 
def intialize_normal_model(ng, data):
    mod_ps = np.repeat(1.0/ng, ng)
    if ng ==2:
        n1,n2 = norm_multdist(ng, data)
        mix_ = mixture.MixtureModel(ng, mod_ps,[n1,n2])
    elif ng == 3:  
        n1,n2,n3 = norm_multdist(ng, data)
        mix_ = mixture.MixtureModel(ng, mod_ps,[n1,n2,n3])
    elif ng == 4:  
        n1,n2,n3,n4 = norm_multdist(ng, data)
        mix_ = mixture.MixtureModel(ng, mod_ps,[n1,n2,n3,n4])
    elif ng == 5:  
        n1,n2,n3,n4,n5 = norm_multdist(ng, data)
        mix_ = mixture.MixtureModel(ng, mod_ps,[n1,n2,n3,n4,n5])
    elif ng == 6:  
        n1,n2,n3,n4,n5,n6 = norm_multdist(ng, data)
        mix_ = mixture.MixtureModel(ng, mod_ps,[n1,n2,n3,n4,n5,n6])
    elif ng == 7:  
        n1,n2,n3,n4,n5,n6,n7 = norm_multdist(ng, data)
        mix_ = mixture.MixtureModel(ng, mod_ps,[n1,n2,n3,n4,n5,n6,n7])
    elif ng == 8:  
        n1,n2,n3,n4,n5,n6,n7,n8 = norm_multdist(ng, data)
        mix_ = mixture.MixtureModel(ng, mod_ps,[n1,n2,n3,n4,n5,n6,n7,n8])
    return mix_ 

# Run Normal Mixture Model
def normal_mixmod(ng, data):
    mix_mod = intialize_normal_model(ng, data)
    mix_dat = build_mix_dat(data)
    mix_mod.modelInitialization(mix_dat)
    mix_mod.EM(mix_dat, 40,.2)
    mix_mod.classify(mix_dat)
    return mix_mod
   
# Results 
def GBFMM(ng,data): 
    out = normal_mixmod(ng, data)
    traj = list()
    for i in range(ng):
        traj.append(out.components[i].distList[0].mu)
    return traj

# Trajectory Plotting Function
def plot_traj(outs):
    if len(outs) == 2:
        d = {'one' : outs[0], 
        'two' : outs[1], 
        'Time': range(1,len(outs[0])+1)}
    elif len(outs) == 3:
        d = {'one' : outs[0], 
        'two' : outs[1], 
        'three' : outs[2], 
        'Time': range(1,len(outs[0])+1)}
    elif len(outs) == 4: 
        d = {'one' : outs[0], 
        'two' : outs[1], 
        'three' : outs[2], 
        'four' : outs[3], 
        'Time': range(1,len(outs[0])+1)}
    elif len(outs) == 5:
        d = {'one' : outs[0], 
        'two' : outs[1], 
        'three' : outs[2], 
        'four' : outs[3], 
        'five' : outs[4], 
        'Time': range(1,len(outs[0])+1)}
    elif len(outs) == 6:
        d = {'one' : outs[0], 
        'two' : outs[1], 
        'three' : outs[2], 
        'four' : outs[3], 
        'five' : outs[4], 
        'six' : outs[5], 
        'Time': range(1,len(outs[0])+1)}
    elif len(outs) == 7: 
        d = {'one' : outs[0],
        'two' : outs[1], 
        'three' : outs[2], 
        'four' : outs[3], 
        'five' : outs[4], 
        'six' : outs[5], 
        'seven' : outs[6], 
        'Time': range(1,len(outs[0])+1)}
    elif len(outs) == 8:
        d = {'one' : outs[0],
        'two' : outs[1], 
        'three' : outs[2], 
        'four' : outs[3], 
        'five' : outs[4], 
        'six' : outs[5], 
        'seven' : outs[6], 
        'eight' : outs[7], 
        'Time': range(1,len(outs[0])+1)}
    dat = pd.DataFrame(d) 
    dm = pd.melt(dat, id_vars=['Time'], var_name='Latent Class', value_name='Value') 
    print ggplot(dm, aes('Time', 'Value', color='Latent Class')) + \
        geom_point(size=80) + \
        geom_line(size=3) + \
        theme_bw()

## 3) Testing

## 4) Application and comparison